In [9]:
import openeo
import pandas as pd

import boto3
import configparser
import os
import urllib3

import folium
import geopandas as gpd

import tempfile

# Python Standard Libraries
import os
import datetime as dt

# Data Manipulation Libraries
import numpy as np
import xarray as xr

# Visualization Libraries
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

import joblib

# Climate Data Store API for retrieving climate data
import cdsapi

import matplotlib.pyplot as plt

import statsmodels.api as sm

urllib3.disable_warnings()

In [3]:
file_name = {}  # dictionary containing [data source : file name]

# Add the data sources and file names
file_name.update({"era5": "temperature_era5_predict.nc"})

# Create the paths to the files
path_to = {
    source: os.path.join(f"/home/eouser/hackathon/data/{source}/", file) for source, file in file_name.items()
}

# Create necessary directories if they do not exist
for path in path_to.values():
    os.makedirs(
        os.path.dirname(path), exist_ok=True
    )  # create the folder if not available

path_to

{'era5': '/home/eouser/hackathon/data/era5/temperature_era5_predict.nc'}

In [4]:

client = cdsapi.Client()

dataset = 'reanalysis-era5-single-levels-monthly-means'
request = {
  'product_type': ['monthly_averaged_reanalysis'],
  'variable': ['2m_temperature', 'land_sea_mask'],
  'year': ['2024'],
  'month': list(range(1, 13)),
  'time': '00:00',
  'data_format': 'netcdf',
  'area': [47, 12, 46, 14],
  'download_format': 'unarchived' 
}
target = path_to['era5']

client.retrieve(dataset, request, target)

2025-03-07 17:11:33,545 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-03-07 17:11:33,556 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-03-07 17:11:33,812 INFO Request ID is b864cc2f-1fcb-4df8-bd93-46fdd25038df
2025-03-07 17:11:33,883 INFO status has been updated to accepted
2025-03-07 17:11:47,576 INFO status has been updated to running
2025-03-07 17:11:55,268 WARNING Structural differences in grib fields detected when opening in xarray. Opening the grib file safely, however this may result in files with non-intuitive filenames.
2025-03-07 17:11:55,269 INFO status has been updated to successful


'/home/eouser/hackathon/data/era5/temperature_era5_predict.nc'

In [10]:

data_24 = xr.open_dataset('/home/eouser/hackathon/data/era5/temperature_era5_predict.nc')
data_24

data = xr.open_dataset('/home/eouser/hackathon/data/era5/temperature_era5.nc')
data


<xarray.Dataset> Size: 32kB
Dimensions:     (valid_time: 84, latitude: 5, longitude: 9)
Coordinates:
    number      int64 8B ...
  * valid_time  (valid_time) datetime64[ns] 672B 2017-01-01 ... 2023-12-01
  * latitude    (latitude) float64 40B 47.0 46.75 46.5 46.25 46.0
  * longitude   (longitude) float64 72B 12.0 12.25 12.5 ... 13.5 13.75 14.0
    expver      (valid_time) <U4 1kB ...
Data variables:
    t2m         (valid_time, latitude, longitude) float32 15kB ...
    lsm         (valid_time, latitude, longitude) float32 15kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts

In [31]:
dat = data_24.t2m[6, :, :]
dat

<xarray.DataArray 't2m' (latitude: 5, longitude: 9)> Size: 180B
[45 values with dtype=float32]
Coordinates:
    number      int64 8B ...
    valid_time  datetime64[ns] 8B 2024-07-01
  * latitude    (latitude) float64 40B 47.0 46.75 46.5 46.25 46.0
  * longitude   (longitude) float64 72B 12.0 12.25 12.5 ... 13.5 13.75 14.0
    expver      <U4 16B ...
Attributes: (12/32)
    GRIB_paramId:                             167
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      45
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            avgua
    ...                                       ...
    GRIB_totalNumber:                         0
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            unknown
    GRIB_surface:                             0.0

In [14]:
data_agg2 = dat.to_numpy()
data_agg2

array([[[267.2566 , 266.4226 , 265.97925, 265.69604, 266.40894,
         267.7898 , 268.2605 , 268.5476 , 269.05347],
        [267.77612, 266.96558, 266.59448, 266.9812 , 268.87964,
         270.30347, 271.18628, 271.6648 , 272.57886],
        [267.93042, 268.03198, 268.65698, 269.46948, 272.2019 ,
         273.1765 , 272.65308, 271.698  , 271.6902 ],
        [269.05933, 271.7019 , 273.6316 , 274.52417, 275.53198,
         275.489  , 274.65112, 272.67847, 271.97144],
        [274.63354, 275.67456, 276.88354, 276.50464, 277.41675,
         277.53784, 277.2683 , 275.50854, 274.59253]],

       [[271.85156, 270.7832 , 270.58594, 270.91602, 271.92773,
         272.9121 , 273.46875, 273.96094, 274.56055],
        [272.77734, 272.50586, 272.50977, 273.04883, 274.14844,
         274.93164, 275.21094, 275.58203, 276.48047],
        [272.77148, 273.36523, 274.21094, 275.1328 , 276.5078 ,
         276.92773, 276.5039 , 276.00195, 276.23047],
        [273.89648, 275.83984, 277.375  , 278.33203, 2

In [18]:
data_agg2_sin = np.sin(data_agg2)

In [24]:
shape = data_agg2_sin.shape

In [19]:
data_agg2_sin_reshape = data_agg2_sin.reshape(-1, 1)

In [25]:
data_ag = data_agg2_sin_reshape.reshape(shape)

In [26]:
months = [i for i in range(1, 13)]
months

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [27]:
months = np.array(months)

months = months.reshape(-1, 1)

In [28]:
months_all_years = months * 1
months_all_years

array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12]])

In [29]:
years = [i for i in range(8, 9)]
years

[8]

In [30]:
l = []
for k in years:
    for i in range(1, 13):
        l.append(k)
l

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]

In [7]:
model = joblib.load('/home/eouser/hackathon/data/regression_model_temp.joblib')

In [38]:
month = 6
dat = data_24.t2m[month, :, :]
data_agg2 = dat.to_numpy()
data_agg2_sin = np.sin(data_agg2)

shape = data_agg2_sin.shape

months = [i for i in range(1, month+1)]
months

data_agg2_sin_reshape = data_agg2_sin.reshape(-1, 1)

months


[1, 2, 3, 4, 5, 6]

In [ ]:
model = sm.OLS(y, data[['months', 'years']])
results = model.fit()
print(results.summary())

In [ ]:
model.predict(data.t2m[:,:,:])